<a href="https://colab.research.google.com/github/FatemehArabzadeh/persian_qa/blob/main/qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset

ds = load_dataset("SajjadAyoubi/persian_qa", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/9008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/930 [00:00<?, ? examples/s]

In [4]:
ds = ds.train_test_split(test_size=0.2)

In [5]:
ds["train"][0]

{'id': 7325,
 'title': 'بسکتبال',
 'context': 'بسکتبال یا توپ سبد ورزشی گروهی است که در آن دو گروه پنج \u200cنفره در یک سالن مستطیل\u200cشکل به بازی می\u200cپردازند. هدف بازی پرتاب توپ به درون سبدهایی به قطر ۴۶ سانتی\u200cمتر و به ارتفاع ۳٫۰۴۸ متر است که در دو سمت زمین قرار دارند و به تخته\u200cای به نام «بک\u200cبُرد» متصل\u200cاند. هر تیم باید توپ را به درون سبد تیم حریف بیندازد. در بازی بسکتبال هر تیم برای کسب امتیاز باید در محدوده پرتاب، توپ را به سمت حلقه پرتاب نماید زمانی که توپ وارد حلقه شد، تیم موفق به کسب دو امتیاز می\u200cشود. زمانی\u200cکه بازیکنی از پشت خط سه امتیازی موفق به وارد کردن توپ درون حلقه شود سه امتیاز برای تیم در نظر گرفته می\u200cشود. مهارت\u200cهایی فردی مانند شوت\u200cکردن دریبل\u200cزدن و ریباند توپ به همراه کارهای تیمی مثل حمله و دفاع، شروط لازم برای موفقیت شرکت\u200cکنندگان در این ورزش می\u200cباشد. این ورزش نخستین\u200cبار توسط دکتر جیمز نای\u200cاسمیت در پائیز ۱۸۹۱ ابداع گردید؛ و نخستین مجموعه رسمی از قوانین بسکتبال توسط وی وضع گردید. بسکتبال به\u200cطور 

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [7]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]

        if len(answer["answer_start"]) > 0 :
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])
        else:
            start_char=0
            end_char=0

        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char :
            start_positions.append(0)
            end_positions.append(0)
        elif end_char == 0 and start_char == 0 :
           start_positions.append(0)
           end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [8]:
tokenized_ds = ds.map(preprocess_function, batched=True, remove_columns=ds["train"].column_names)

Map:   0%|          | 0/7206 [00:00<?, ? examples/s]

Map:   0%|          | 0/1802 [00:00<?, ? examples/s]

In [9]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [10]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("google-bert/bert-base-multilingual-uncased")

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at google-bert/bert-base-multilingual-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = TrainingArguments(
    output_dir="bert-base-multilingual-uncased-persian-qa",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.113806
2,2.769600,1.973702
3,1.746100,2.027152


TrainOutput(global_step=1353, training_loss=2.038766820080088, metrics={'train_runtime': 1828.2938, 'train_samples_per_second': 11.824, 'train_steps_per_second': 0.74, 'total_flos': 4236534965339136.0, 'train_loss': 2.038766820080088, 'epoch': 3.0})

In [12]:
trainer.push_to_hub()


events.out.tfevents.1723198980.9ea8c52e07ba.400.0:   0%|          | 0.00/6.67k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Farabzadeh/bert-base-multilingual-uncased-persian-qa/commit/cc45e71ae43a3e140c59e57d9d164349866f417f', commit_message='End of training', commit_description='', oid='cc45e71ae43a3e140c59e57d9d164349866f417f', pr_url=None, pr_revision=None, pr_num=None)

# Inference

In [15]:
question = "گردو شامل آموزش چند زبان برنامه نویسی می باشد ؟"
context = "گردو شامل آموزش 50 زبان برنامه نویسی و 90 آموزش برنامه ی کاربردی می باشد "

In [16]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="bert-base-multilingual-uncased-persian-qa")
question_answerer(question=question, context=context)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'score': 0.27343520522117615,
 'start': 16,
 'end': 36,
 'answer': '50 زبان برنامه نویسی'}

# Manually

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased-persian-qa")
inputs = tokenizer(question, context, return_tensors="pt")

In [18]:
import torch
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("bert-base-multilingual-uncased-persian-qa")
with torch.no_grad():
    outputs = model(**inputs)

In [19]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [22]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'50 زبان برنامه نویسی'

# New Section